In [1]:
import gzip
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from PIL import Image

In [2]:
train = gzip.open('train-images-idx3-ubyte.gz', 'rb')
train_label = gzip.open('train-labels-idx1-ubyte.gz', 'rb')
test = gzip.open('t10k-images-idx3-ubyte.gz', 'rb')
test_label = gzip.open('t10k-labels-idx1-ubyte.gz', 'rb')

train.read(16)
train_label.read(8)
test.read(16)
test_label.read(8)

train_image_data = np.frombuffer(train.read(), dtype=np.uint8).reshape(-1, 28, 28)
train_label_data = np.frombuffer(train_label.read(), dtype=np.uint8)

test_image_data = np.frombuffer(test.read(), dtype=np.uint8).reshape(-1, 28, 28)
test_label_data = np.frombuffer(test_label.read(), dtype=np.uint8)

# Ensure train_image_data and train_label_data have compatible shapes
train_image_data = train_image_data[:len(train_label_data)] # This will keep the first 12000 entries from the image data, or
# train_label_data = train_label_data[:len(train_image_data)] # will keep the first 48000 labels, based on your desired approach.

train_images, val_images, train_labels, val_labels = train_test_split(
    train_image_data, train_label_data, test_size=0.2, random_state=42 # Added random_state for reproducibility.
)

In [3]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_val_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images.numpy().astype(np.uint8)
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx])
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# Convert data to PyTorch tensors
train_images = torch.tensor(train_images, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)  # Labels should be long tensors
val_images = torch.tensor(val_images, dtype=torch.float32)
val_labels = torch.tensor(val_labels, dtype=torch.long)
test_image_data = torch.tensor(test_image_data, dtype=torch.float32)
test_label_data = torch.tensor(test_label_data, dtype=torch.long)

# Create custom datasets with transforms
train_dataset = CustomDataset(train_images, train_labels, transform=transform_train)
val_dataset = CustomDataset(val_images, val_labels, transform=transform_val_test)
test_dataset = CustomDataset(test_image_data, test_label_data, transform=transform_val_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc1 = nn.Linear(64 * 6 * 6, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 64 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = CNN()

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [9]:
for epoch in range(20):
    model.train()
    for images, labels in train_loader:
        images = images.float()
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.float()
            output = model(images)
            val_loss += criterion(output, labels).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(labels.view_as(pred)).sum().item()
    val_loss /= len(val_loader.dataset)

    print('Epoch: {} \tValidation Loss: {:.6f} \tValidation Accuracy: {:.2f}%'.format(epoch + 1, val_loss, 100. * correct / len(val_loader.dataset)))

Epoch: 1 	Validation Loss: 0.006341 	Validation Accuracy: 85.68%
Epoch: 2 	Validation Loss: 0.005268 	Validation Accuracy: 87.62%
Epoch: 3 	Validation Loss: 0.004900 	Validation Accuracy: 88.54%
Epoch: 4 	Validation Loss: 0.004647 	Validation Accuracy: 89.01%
Epoch: 5 	Validation Loss: 0.004392 	Validation Accuracy: 89.74%
Epoch: 6 	Validation Loss: 0.004272 	Validation Accuracy: 90.02%
Epoch: 7 	Validation Loss: 0.004156 	Validation Accuracy: 90.16%
Epoch: 8 	Validation Loss: 0.004169 	Validation Accuracy: 90.12%
Epoch: 9 	Validation Loss: 0.003968 	Validation Accuracy: 90.50%
Epoch: 10 	Validation Loss: 0.004249 	Validation Accuracy: 89.89%
Epoch: 11 	Validation Loss: 0.003917 	Validation Accuracy: 90.84%
Epoch: 12 	Validation Loss: 0.003817 	Validation Accuracy: 91.03%
Epoch: 13 	Validation Loss: 0.003752 	Validation Accuracy: 91.02%
Epoch: 14 	Validation Loss: 0.003744 	Validation Accuracy: 91.24%
Epoch: 15 	Validation Loss: 0.003638 	Validation Accuracy: 91.47%
Epoch: 16 	Validati

Sources:
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html